In [ ]:
# --- Setup: clone repo so validator.py + sample_images are available ---
REPO_URL = "https://github.com/slavebotallen/IndianFlagValidator.git"
REPO_DIR = REPO_URL.rsplit("/", 1)[-1].replace(".git","")

!rm -rf $REPO_DIR
!git clone $REPO_URL
%cd $REPO_DIR

# If sample_images.zip exists, unzip it
import os
if os.path.exists("sample_images.zip"):
    !unzip -o sample_images.zip -d sample_images

# Import your validator module
from validator import validate


# Indian_Flag_Validator_Submission.ipynb - Colab

Author: Rithik Srivastava  
Student Code: IITRPRAI_24082046

---

How to Run

1. Upload **sample_images.zip** in the left panel (Files).  
2. Run the cells top to bottom.  
3. Reports will be saved to the `outputs/` folder and zipped as `outputs.zip`.  
4. The summary table shows key results for each test image.


In [ ]:
!python --version
import os, glob, json, time, zipfile
import pandas as pd
from google.colab import files
from validator import validate

In [ ]:
# 1) Unzip uploaded sample_images.zip
!unzip -o sample_images.zip -d sample_images

In [ ]:
# 2) Paths
SAMPLE_DIR = "sample_images"
OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

In [ ]:
# 3) Process images
paths = sorted([p for p in glob.glob(SAMPLE_DIR + "/**/*", recursive=True) if os.path.isfile(p)])
print("Found images:", len(paths))

rows = []
for p in paths:
    try:
        t0 = time.time()
        rep = validate(p)
        elapsed = time.time() - t0

        base = os.path.splitext(os.path.basename(p))[0]
        outp = os.path.join(OUT_DIR, base + "_report.json")
        with open(outp, "w") as f:
            json.dump(rep, f, indent=2)

        # Summary row (fixed typo)
        rows.append({
            "Image": os.path.basename(p),
            "Processed (s)": round(elapsed, 3),
            "Aspect Status": rep["aspect_ratio"]["status"],
            "Aspect Actual": rep["aspect_ratio"]["actual"],
            "Stripe Status": rep["stripe_proportion"]["status"],
            "Top Stripe": rep["stripe_proportion"]["top"],
            "Middle Stripe": rep["stripe_proportion"]["middle"],
            "Bottom Stripe": rep["stripe_proportion"]["bottom"],
            "Chakra Pos Status": rep["chakra_position"]["status"],
            "Spokes Detected": rep["chakra_spokes"]["detected"],
            "Spokes Status": rep["chakra_spokes"]["status"],
            # Color statuses & deviations
            "Saffron": f"{rep['colors']['saffron']['status']} ({rep['colors']['saffron']['deviation']})",
            "White": f"{rep['colors']['white']['status']} ({rep['colors']['white']['deviation']})",
            "Green": f"{rep['colors']['green']['status']} ({rep['colors']['green']['deviation']})",
            "Chakra Blue": f"{rep['colors']['chakra_blue']['status']} ({rep['colors']['chakra_blue']['deviation']})",
        })

        print(f"✅ {os.path.basename(p)} ->", json.dumps(rep))

        # Optional: warn if processing time > 3s
        if elapsed > 3.0:
            print(f"⚠ WARNING: Processing time exceeded 3 seconds for {os.path.basename(p)}: {elapsed:.3f}s")

    except Exception as e:
        print(f"❌ {os.path.basename(p)} ->", e)

df = pd.DataFrame(rows)
display(df)

In [ ]:
# 4) Zip outputs for download
with zipfile.ZipFile("outputs.zip", "w", zipfile.ZIP_DEFLATED) as zf:
    for p in glob.glob(os.path.join(OUT_DIR, "*.json")):
        zf.write(p)

files.download("outputs.zip")